In [6]:
import json
import statistics
from collections import Counter


In [9]:
# Load the JSON file
def load_json(filepath):
    with open(filepath, 'r') as f:
        data = json.load(f)
    return data

# Compute statistics
def compute_keypoint_statistics(data):
    keypoint_counts = [len(model['keypoints']) for model in data]
    
    total_models = len(keypoint_counts)
    total_keypoints = sum(keypoint_counts)
    average_keypoints = statistics.mean(keypoint_counts) if keypoint_counts else 0
    min_keypoints = min(keypoint_counts) if keypoint_counts else 0
    max_keypoints = max(keypoint_counts) if keypoint_counts else 0
    value_counts = Counter(keypoint_counts)
    
    stats = {
        'Total models': total_models,
        'Average keypoints per model': average_keypoints,
        'Minimum keypoints in any model': min_keypoints,
        'Maximum keypoints in any model': max_keypoints,
        'Value counts': value_counts
    }
    
    return stats

In [10]:
filepath = 'cap.json' 
data = load_json(filepath)
stats = compute_keypoint_statistics(data)

print("Summary Statistics:")
for k, v in stats.items():
    print(f"{k}: {v}")


Summary Statistics:
Total models: 38
Average keypoints per model: 5.947368421052632
Minimum keypoints in any model: 5
Maximum keypoints in any model: 6
Value counts: Counter({6: 36, 5: 2})
